A single-class object detector was built based on yolov3.

$ sbatch -o out.pillow_train -N 1 -n 1 -p gpu -t 02-00:00:00 --mem=20g  --qos gpu_access --gres=gpu:1 --wrap "./darknet detector train dataset/obj.data custom/yolov3_pillow.cfg darknet53.conv.74 -gpu 0 > yolov3_train.log"
Submitted batch job 25323682

Trained 500 labeled images on 6000 iterations. Average loss = 0.081272
Tested on 125 test images and average IoU=76.70%, recall = 88.2%.

Then sent all 1555 images for detetion, including the 625 traing and test sets.

$ sbatch -o out.pillow_ownYolo -N 1 -n 1 -p gpu -t 12:00:00 --qos gpu_access --gres=gpu:1 --mem=16g --wrap "./darknet detector test dataset/obj.data dataset/yolov3_pillow_det.cfg dataset/yolov3_pillow_final.weights -ext_output < ../Pillow_select_list > Pillow_select_Own_results.txt"

Default threshold 25%.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import compress
from ast import literal_eval
from PIL import Image
dir = os.getcwd()
home = os.path.dirname(os.path.dirname(dir)) ## 2 levels up of working directory dir

In [2]:
# path_input_image = r'Yolov3_pillow/Pillow_select_list'
# path_cropped_image = r'Yolov3_pillow/NewPrediction/result_img'
# path_output_log = r'Yolov3_pillow/NewPrediction/Pillow_select_Own_results.txt'

def getbox (path_input_image, path_cropped_image, path_output_log):

    ##(part 1) 
    # use this format for all path: path = r'path/to/happiness'
    # import an index file that has the order in which the catalog images are detected on darknet.
    pd_index_darknet = pd.read_csv(os.path.join(home, path_input_image), sep = " ", header = None)
    pd_index_darknet.columns = ["image"]
    pd_index_darknet["dn_order"] = pd_index_darknet.index.values + 1
    
    t1 = [i[1] for i in pd_index_darknet['image'].apply(os.path.split)]
    pd_index_darknet['index_image'] = [os.path.splitext(i)[0] for i in t1]
    del t1
    
    ##(part 2) 
    # list of darknet output cropped images
    # these images named by dn_order (1 per input image), box id (sequential of all detected
    dn_crop = os.listdir(os.path.join(home, path_cropped_image))
    
    # for example, "img_184_379_465_Furniture.jpg.jpg": dn_order = 184, box_order =379, class id #465 for furniture
    class_name = [str(os.path.splitext(os.path.splitext(i)[0])[0]).split('_')[-1] for i in dn_crop]
    dn_order = [str(os.path.splitext(os.path.splitext(i)[0])[0]).split('_')[1] for i in dn_crop]
    box_order = [str(os.path.splitext(os.path.splitext(i)[0])[0]).split('_')[2] for i in dn_crop]
    class_id = [str(os.path.splitext(os.path.splitext(i)[0])[0]).split('_')[-2] for i in dn_crop]
    t1 = pd.DataFrame({"dn_crop": dn_crop,
                        "dn_order": list(map(int,dn_order)),
                        "box_order": list(map(int,box_order)),
                        "class_name": class_name,
                        "class_id": list(map(int,class_id))})
    crop = pd.merge(t1, pd_index_darknet, on='dn_order', how='left').sort_values(by=['box_order'])
    crop.index = crop['box_order']
    
    ##(part 3)
    # Next, import the detection log file to get coordinates for each bounding box
    # Bounding box (left_x, top_y, width, height)
    f=open(os.path.join(home, path_output_log), "r")
    if f.mode == 'r':
        log =f.readlines()
    #print("The log has " + str(len(log)) + " lines")

    # extract lines with box coordinates:
    log_w_box = list(compress(log, ["(" in i for i in log]))
    #print("Extracted " + str(len(log_w_box)) + " lines, each with a box coordinate.")

    # extract coordinates of bounding boxes
    box_coordinate = [i.split("(")[1].split(")")[0] for i in log_w_box]
   
    ##(part 4)
    # This is very annoying, but when darknet saved cropped images for a given input, 
    # the output image names are not following the same order as in log, despite what's claimed. 
    # Bounding boxes in the log was ordered by greatest prob% of detected objects in that bounding box.
    # So the plan is to get the size of cropped images and try to match up with the bounding box measurements.    
    folderpath = os.path.join(home, path_cropped_image)

    def getsize(image_name):
        with Image.open(os.path.join(folderpath,image_name)) as img:
            width, height = img.size
        return [width, height, width*height]

    size = [getsize(i) for i in dn_crop]

    pd_size = pd.DataFrame(size, columns=["crop_width","crop_height","crop_area"])
    pd_size ["dn_crop"] = dn_crop

    crop_mad = pd.merge(crop, pd_size, on='dn_crop', how='left').sort_values(by=['box_order'])

    def str_to_int(string):
        l = []
        for t in string.split():
            try:
                l.append(int(t))
            except ValueError:
                pass
        return l
    t2 = [str_to_int(i) for i in box_coordinate]
    t4 = pd.DataFrame.from_records(t2, columns=['left_x', 'top_y', 'width', 'height'])
    # t4 is the original order of bounding boxes from log file.
    # now add dn_order which indicates which croppe image this measurements refer to.
    t4['dn_order'] = crop["dn_order"].values.tolist()
    t4['box_area'] = np.array(t4['width']) * np.array(t4['height'])

    ##(part 4)
    # sort both t4 and crop_mad by dn_order and then box_area
    # then merge two
    t4_sorted = t4.sort_values(by=['dn_order','box_area']).reset_index(drop=True)
    crop_mad_sorted = crop_mad.sort_values(by=['dn_order','crop_area']).reset_index(drop=True) 

    crop_happy = crop_mad_sorted.join(t4_sorted.iloc[:,range(4)]).sort_values(by=['box_order']).reset_index(drop=True)
    
    return crop_happy

In [ ]:
path_input_image = r'Yolov3_pillow/Pillow_select_list'
path_cropped_image = r'Yolov3_pillow/NewPrediction/result_img'
path_output_log = r'Yolov3_pillow/NewPrediction/Pillow_select_Own_results.txt'

In [3]:
output = getbox(path_input_image = r'Yolov3_pillow/Pillow_select_list',
             path_cropped_image = r'Yolov3_pillow/NewPrediction/result_img',
             path_output_log = r'Yolov3_pillow/NewPrediction/Pillow_select_Own_results.txt')
print(str(output.shape[0]) + " bounding boxes returned from detection.")
output.tail(5)

1812 bounding boxes returned from detection.


,dn_crop,dn_order,box_order,class_name,class_id,image,index_image,crop_width,crop_height,crop_area,left_x,top_y,width,height
1807,img_1552_1808_0_Pillow.jpg.jpg,1552,1808,Pillow,0,Pillow_select/Page_9_6.jpg,Page_9_6,448,431,193088,16,27,448,431
1808,img_1553_1809_0_Pillow.jpg.jpg,1553,1809,Pillow,0,Pillow_select/Page_9_7.jpg,Page_9_7,417,370,154290,39,57,417,371
1809,img_1554_1810_0_Pillow.jpg.jpg,1554,1810,Pillow,0,Pillow_select/Page_9_8.jpg,Page_9_8,235,201,47235,218,138,235,201
1810,img_1554_1811_0_Pillow.jpg.jpg,1554,1811,Pillow,0,Pillow_select/Page_9_8.jpg,Page_9_8,376,276,103776,54,113,376,276
1811,img_1555_1812_0_Pillow.jpg.jpg,1555,1812,Pillow,0,Pillow_select/Page_9_9.jpg,Page_9_9,364,356,129584,57,67,363,357


In [6]:
# assign two scores: a deviation score and a centerness score
# dev_score = d/sqrt(a); c_score = d 
# where d = distance between center of input image and center of box
# and a = area of box
def get_score (list):
    left_x = list[0]
    top_y = list[1]
    w = list[2]
    h = list[3]
    d = np.math.sqrt((left_x + w/2)**2+(top_y - h/2)**2)
    sr_area = np.math.sqrt(w * h)
    return [d/sr_area, d]

scores = output[['left_x','top_y','width','height']].apply(get_score, axis = 1)
output ['deviation-score'] = [i[0] for i in scores]
output ['centerness-score'] = [i[1] for i in scores]

In [7]:
output.tail(5)

,dn_crop,dn_order,box_order,class_name,class_id,image,index_image,crop_width,crop_height,crop_area,left_x,top_y,width,height,deviation-score,centerness-score
1807,img_1552_1808_0_Pillow.jpg.jpg,1552,1808,Pillow,0,Pillow_select/Page_9_6.jpg,Page_9_6,448,431,193088,16,27,448,431,0.694500,305.175769
1808,img_1553_1809_0_Pillow.jpg.jpg,1553,1809,Pillow,0,Pillow_select/Page_9_7.jpg,Page_9_7,417,370,154290,39,57,417,371,0.709001,278.870041
1809,img_1554_1810_0_Pillow.jpg.jpg,1554,1810,Pillow,0,Pillow_select/Page_9_8.jpg,Page_9_8,235,201,47235,218,138,235,201,1.553305,337.589247
1810,img_1554_1811_0_Pillow.jpg.jpg,1554,1811,Pillow,0,Pillow_select/Page_9_8.jpg,Page_9_8,376,276,103776,54,113,376,276,0.755217,243.287895
1811,img_1555_1812_0_Pillow.jpg.jpg,1555,1812,Pillow,0,Pillow_select/Page_9_9.jpg,Page_9_9,364,356,129584,57,67,363,357,0.731349,263.276471


In [5]:
print("Detected " + str(len(list(Counter(output['dn_order']).keys())))
      + " out of the 1555 images.")

Detected 1549 out of the 1555 images.


In [8]:
export_csv = output.to_csv (os.path.join(dir,r'darknet_detected_cropped.csv'), 
                                             index = True, 
                                             header = True)